In [4]:
from services.score import Score
from services import resume_scraper
from db_tools import get_job_desc

resume_path = 'data\\resumes\\Jackson_Giemza.pdf'
# raw_resume = resume_scraper.extract_text_from_pdf(resume_path)
raw_resume = resume_scraper.extract_text_from_pdf('data\\resumes\\EduardoBenjaminResume-1Page-3.pdf')
raw_jd = get_job_desc('1325')

scorer = Score(raw_resume, raw_jd)
print(scorer.compute_similarity())

tensor([[0.5572]])
